In [ ]:
GSE_Train = "GSE63990"
target = "infection_status" #目标存的列。暂时还都不用改。

In [ ]:
# 各种库导入
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
import warnings
import GEOparse
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LinearRegression, RidgeClassifier, Ridge, Lasso, LassoCV, LogisticRegression
from sklearn.svm import SVC, LinearSVC, NuSVC
warnings.filterwarnings('ignore')

In [ ]:
# 数据导入 及 预览数据. 输出末2行
gse = GEOparse.get_GEO(geo=GSE_Train, destdir="./datasets", silent=True)
gpls = gse.metadata['platform_id']
gpl = GEOparse.get_GEO(geo=gpls[0], destdir="./datasets", silent=True)
gse_csv = pd.read_csv('./datasets/' + GSE_Train + '.csv')
gse_csv.tail(2) # 预览数据

In [ ]:
# 选择的模型
up_feature =  ['ISG15', 'MX1', 'CD74']	
up_weight =  [12, 9, 25]		
up_times = 88822.416610
up_intercept = -3.835824358628409
up_model = LinearRegression()
down_feature =  ['CD177', 'IL1R2', 'S100A12', 'MMP9']
down_weight = [15, 9, 24, 1]
down_times = 876540.757881
down_model = LogisticRegression()
down_intercept = 0.00400899

# 实际输入的模型
feature =  up_feature + down_feature
print('feature:', feature)
weight = up_weight + (np.array(down_weight) * -1).tolist()
print('weight:', weight)

In [ ]:
# 仅选择特征和最终结果. 输出末2行
ID = []
for i in range(len(feature)):
    ID.append(gpl.table[gpl.table['Gene Symbol'] == feature[i]]['ID'].values[0])
ID.append(target)
ID = np.array(ID).reshape(-1)
testdata = gse_csv[ID]
# 把testdata的columns的名字改成feature
testdata.columns = feature + ["target"]
testdata.tail(2)

In [ ]:
# 计算每一种基因在不同感染情况下的表达量的平均值
testdata_mean = testdata.groupby("target").mean()
testdata_mean

In [ ]:
# 对平均值都乘以权重
testdata_mean_weight = testdata_mean * weight
testdata_mean_weight

In [ ]:
up_ID = []
for i in range(len(up_feature)):
    up_ID.append(gpl.table[gpl.table['Gene Symbol'] == up_feature[i]]['ID'].values[0])
up_ID = np.array(up_ID).reshape(-1)
up_ID.astype('str')
up_testdata = gse_csv[up_ID]
up_testdata.columns = up_feature
up_testdata_weight = up_testdata * up_weight
up_testdata_weight_sum = up_testdata_weight.sum(axis=1)
up_testdata_weight_z = up_testdata_weight_sum / up_times + up_intercept
up_testdata_weight_score = 1 / (1 + np.exp(-up_testdata_weight_z))

In [ ]:
up_testdata_weight_score

In [ ]:
up_ID = []
for i in range(len(up_feature)):
    up_ID.append(gpl.table[gpl.table['Gene Symbol'] == up_feature[i]]['ID'].values[0])
up_ID = np.array(up_ID).reshape(-1)
up_ID.astype('str')
up_testdata = gse_csv[up_ID]
up_testdata.columns = up_feature
up_testdata_weight = up_testdata * up_weight
up_testdata_weight_sum = up_testdata_weight.sum(axis=1)
up_testdata_weight_z = up_testdata_weight_sum / up_times + up_intercept
up_testdata_weight_score = 1 / (1 + np.exp(-up_testdata_weight_z))

down_ID = []
for i in range(len(down_feature)):
    down_ID.append(gpl.table[gpl.table['Gene Symbol'] == down_feature[i]]['ID'].values[0])
down_ID = np.array(down_ID).reshape(-1)
down_ID.astype('str')
down_testdata = gse_csv[down_ID]
down_testdata.columns = down_feature
down_testdata_weight = down_testdata * down_weight
down_testdata_weight_sum = down_testdata_weight.sum(axis=1)
down_testdata_weight_score = down_testdata_weight_sum / down_times + down_intercept
#down_testdata_weight_score = 1 / (1 + np.exp(-down_testdata_weight_z)) # sigmoid

# target
target = gse_csv['infection_status']

# merge
merge = pd.concat([up_testdata_weight_score, down_testdata_weight_score, target], axis=1)
merge.columns = ['up', 'down', 'target']

# plot the figure
# x = up_testdata_weight_with_sum's sum
# y = down_testdata_weight_with_sum's sum
# color = target


# 画图
plt.figure(figsize=(10, 10))
sns.scatterplot(x='up', y='down', hue='target', data=merge)


plt.plot([0, 1], [0, 1], color='red', linewidth=2)
# x=0.5, y=0.5的直线
plt.plot([0.5, 0.5], [0, 1], color='red', linewidth=2)
plt.plot([0, 1], [0.5, 0.5], color='red', linewidth=2)

# count the number of points in each area
plt.plot([0.5, 0.5], [0, 1], color='red', linewidth=2)
plt.plot([0, 1], [0.5, 0.5], color='red', linewidth=2)

# count the number of points in each area
upright = merge[(merge['up'] >= 0.5) & (merge['down'] >= 0.5)]
upleft = merge[(merge['up'] < 0.5) & (merge['down'] >= 0.5)]
downright = merge[(merge['up'] >= 0.5) & (merge['down'] < 0.5)]
downleft = merge[(merge['up'] < 0.5) & (merge['down'] < 0.5)]

# 画出四个区域的点的个数
plt.text(0.6, 0.6, upright.shape[0], fontsize=15)
plt.text(0.4, 0.6, upleft.shape[0], fontsize=15)
plt.text(0.6, 0.4, downright.shape[0], fontsize=15)
plt.text(0.4, 0.4, downleft.shape[0], fontsize=15)

mark_as_up = merge[(merge['up'] >= merge['down'])]
mark_as_down = merge[(merge['up'] < merge['down'])]

plt.text(0.8, 0.2, mark_as_up.shape[0], fontsize=15)
plt.text(0.2, 0.8, mark_as_down.shape[0], fontsize=15)


In [ ]:
# 进行统计
# 分为3个区域：左下角，左上角，右下角，其中右上角的点分到左上角和右下角里
# 左下角：up和down都小于0.5
# 左上角：down大于0.5，且up<down
# 右下角：up大于0.5，且up>down

type1 = merge[(merge['up'] < 0.5) & (merge['down'] < 0.5)]
type2 = merge[(merge['down'] >= 0.5) & (merge['up'] < merge['down'])]
type3 = merge[(merge['up'] >= 0.5) & (merge['up'] > merge['down'])]

# 他们应该对应的target的值
# type1: non-infectious illness	
# type2: bacterial
# type3: viral
# bacterial 的准确率
print('bacterial 的准确率：', type2[type2['target'] == 'bacterial'].shape[0] / merge.shape[0])
print('viral 的准确率：', type3[type3['target'] == 'viral'].shape[0] / merge.shape[0])
print('non-infectious illness 的准确率：', type1[type1['target'] == 'non-infectious illness'].shape[0] / merge.shape[0])

# 那么如果我们只进行二分，y=x的直线左边的点都是bacterial，右边的点都是viral
type4 = merge[(merge['up'] < merge['down'])]
type5 = merge[(merge['up'] >= merge['down'])]
bac_count = merge.groupby('target').count().loc['bacterial', 'up']
vir_count = merge.groupby('target').count().loc['viral', 'up']
print('bacterial 的准确率：', type4[type4['target'] == 'bacterial'].shape[0] / bac_count)
print('viral 的准确率：', type5[type5['target'] == 'viral'].shape[0] / vir_count)

In [ ]:
merge.groupby('target').count()

In [ ]:
# 后面就是画图的部分了。 具体可以看 https://seaborn.pydata.org/tutorial.html
# 直接问ai也可以。 比如 https://www.phind.com/
# https://seaborn.pydata.org/tutorial/axis_grids.html 比如这个下面的几个例子都可以参考
# fig, ax = plt.subplots(1,2, figsize=(20, 10)) # 画布大小横为20，纵为10，一共1行2列
# plt.subplots_adjust(wspace=0.5, hspace=0.5) # 调整子图间距

# testdata_values_melt = pd.melt(testdata, id_vars=['target'], value_vars=feature) # Only keep randomly 100 rows
testdata_values_melt = pd.melt(testdata, id_vars=['target'], value_vars=feature)
# Only keep randomly 100 rows
testdata_values_melt = testdata_values_melt.sample(n=150, random_state=1)
sns.catplot(x="variable", y="value", hue="target", data=testdata_values_melt, ax=ax[0], kind="swarm", # 透明度0.5
            palette="Set2", linewidth=1, edgecolor='gray', size=7, aspect=2)

In [ ]:
data1=testdata_mean.loc[['bacterial','viral']]
data2=testdata_mean_weight.loc[['bacterial','viral']]
data1.plot.bar(stacked=True,width=0.4)
data2.plot.bar(stacked=True,width=0.4,bottom=0)
plt.title('')
plt.show()